# Instalação e importação das bibliotecas

In [ ]:
!pip install opencv-python-headless numpy tensorflow matplotlib

In [ ]:
import os
import cv2
import shutil
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers, optimizers, callbacks

**Atenção:** O aviso acima apenas informa que não foram encontrados drivers cuda e por conta disso será usada a CPU da máquina ao invés da GPU

# Coleta e processamento de dados

In [ ]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0
    return img_normalized


## Balanceamento do DataSet
Para evitar maior peso em um determinado tipo de dado, foi feito um balanceamento do dataset.

Para isso, foi utilizado o método de reamostragem chamado de undersampling, no qual foi selecionado aleatoriamente os dados de uma classe e duplicado até que a quantidade de dados de cada classe se iguale.

In [ ]:
dataset_dir = './dataset'
balanced_dataset_dir = './balanced-dataset'

classes = ['healthy', 'kyphosis', 'lordosis']

os.makedirs(balanced_dataset_dir, exist_ok=True)

image_counts = {}
for cls in classes:
    cls_path = os.path.join(dataset_dir, cls)
    image_counts[cls] = len(os.listdir(cls_path))

max_images = max(image_counts.values())
new_image_counts = {}

for cls in classes:
    os.makedirs(os.path.join(balanced_dataset_dir, cls), exist_ok=True)
    
    images = os.listdir(os.path.join(dataset_dir, cls))
    
    for index, image in enumerate(images):
        new_name = f"{cls}_{index + 1:03d}.jpg"
        shutil.copy(os.path.join(dataset_dir, cls, image), 
                    os.path.join(balanced_dataset_dir, cls, new_name))
    
    new_image_counts[cls] = len(images)

    current_count = new_image_counts[cls]
    while current_count < max_images:
        image_to_copy = random.choice(images)
        
        new_name = f"{cls}_{current_count + 1:03d}.jpg"
        shutil.copy(os.path.join(dataset_dir, cls, image_to_copy),
                    os.path.join(balanced_dataset_dir, cls, new_name))
        
        current_count += 1

print("Dataset balanceado criado em:", balanced_dataset_dir)


### Criando CNN (rede neural convolucional)
Essa função cria um modelo CNN que passa por algumas camadas convolucionais para extrair características das imagens, nas quais foram utilizados as funções:
- ReLU 
	- Função de ativação que permite que a rede neural aprenda padrões complexos
- Pooling
	- Operação de amostragem usada em redes convolucionais para reduzir as dimensões e/ou tamanho da imagem ou das saídas das camadas convolucionais, porém, preservando suas características mais importantes
- Normalização de Batch
	- Normalização de dados de entrada para que a rede neural possa aprender mais rapidamente
- Softmax
	- Função de ativação que converte um vetor de valores reais em uma distribuição de probabilidade

In [ ]:
def create_3_class_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(3, (1, 1), padding="same")(inputs)
    
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    x = base_model(x)
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(3, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model

def create_2_class_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(3, (1, 1), padding="same")(inputs)
    
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    x = base_model(x)
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model

input_shape = (224, 224, 1)

generalist_cnn_model = create_3_class_cnn(input_shape)
lordosis_cnn_model = create_2_class_cnn(input_shape)
kiphosis_cnn_model = create_2_class_cnn(input_shape)

lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)


## Gerador de alterações nas imagens

Para melhorar a amplitude do treinamento, foi utilizado o gerador de alterações nas imagens, que trás alterações em rotação, deslocamento, zoom, brilho e direção.

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    brightness_range = [0.8, 1.2],
    validation_split = 0.3
)

# Pré-processamento dos modelos

In [ ]:
generalist_classes = ['healthy', 'kyphosis', 'lordosis']

generalist_train_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'training'
)

generalist_validation_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'validation'
)

generalist_cnn_model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
lordosis_classes = ['healthy', 'lordosis']

lordosis_train_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'training',
    classes = lordosis_classes
)

lordosis_validation_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'validation',
    classes = lordosis_classes
)

lordosis_cnn_model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
kiphosis_classes = ['healthy', 'kyphosis']

kiphosis_train_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'training',
    classes = kiphosis_classes
)

kiphosis_validation_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'validation',
    classes = kiphosis_classes
)

kiphosis_cnn_model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

# Treinamento do modelo

In [ ]:
generalist_history = generalist_cnn_model.fit(
    generalist_train_generator,
    validation_data = generalist_validation_generator,
    epochs = 30,
    callbacks = [lr_scheduler]
)

lordosis_history = lordosis_cnn_model.fit(
    lordosis_train_generator,
    validation_data = lordosis_validation_generator,
    epochs = 30,
    callbacks = [lr_scheduler]
)

kiphosis_history = kiphosis_cnn_model.fit(
    kiphosis_train_generator,
    validation_data = kiphosis_validation_generator,
    epochs = 30,
    callbacks = [lr_scheduler]
)

## Porque resultados sempre são diferentes:
- Ao inicializar o modelo os pesos da rede neural são inicializados de forma aleatória, e como o processo de otimização do modelo começa a partir de diferentes pontos cada execução pode levar a resultados diferentes, por isso, cada vez que o código for executado os resultados não serão exatamente os mesmos, mas aproximados.
- Image Augmentation: técnicas de rotação, deslocamento, zoom e etc. aplicadas na imagem para criar novas variações para realizar o treinamento do modelo, sendo também um processo aleatório.
- Shuffling: Durante o treinamento, os dados de treino são embaralhados a cada época. Isso garante que o modelo não aprenda de forma dependente da ordem dos exemplos, mas também pode fazer com que os resultados variem.

# Validação dos modelos

In [ ]:
generalist_val_loss, generalist_val_acc = generalist_cnn_model.evaluate(generalist_validation_generator)
lordosis_val_loss, lordosis_val_acc = lordosis_cnn_model.evaluate(lordosis_validation_generator)
kiphosis_val_loss, kiphosis_val_acc = kiphosis_cnn_model.evaluate(kiphosis_validation_generator)

print(f"Validação modelo generalista - Loss: {generalist_val_loss}, Acurácia: {generalist_val_acc}")
print(f"Validação modelo de lordose - Loss: {lordosis_val_loss}, Acurácia: {lordosis_val_acc}")
print(f"Validação modelo de cifose - Loss: {kiphosis_val_loss}, Acurácia: {kiphosis_val_acc}")

# Função de predição

In [ ]:
def predict_image(image_path, model):
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    return predicted_class, prediction

generalist_predicted_class, generalist_result = predict_image('./test-data/001.jpg', generalist_cnn_model)
lordosis_predicted_class, lordosis_result = predict_image('./test-data/001.jpg', lordosis_cnn_model)
kiphosis_predicted_class, kiphosis_result = predict_image('./test-data/001.jpg', kiphosis_cnn_model)

print(f'Predição (modelo generalista): {generalist_result}')
print(f'Classe prevista (modelo generalista): {generalist_classes[generalist_predicted_class]}')

print(f'Predição (modelo de lordose): {lordosis_result}')
print(f'Classe prevista (modelo de lordose): {lordosis_classes[lordosis_predicted_class]}')

print(f'Predição (modelo de cifose): {kiphosis_result}')
print(f'Classe prevista (modelo de cifose): {kiphosis_classes[kiphosis_predicted_class]}')